In [1]:
import numpy as np
import pandas as pd

from src import utils, models, metrics

In [2]:
# read data
df_movies, df_users, df_ratings = utils.read_pickles("../../data/ml-1m-after_eda/")

# split data
train, test = utils.TrainTestSplitter.split_by_percent(df_ratings, 0.8)
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (800167, 6)
Test shape: (200042, 6)


In [6]:
# get true
true_scores = test["Rating"]

# get predict
base_model = models.BaseModelAverage()
base_model.fit(train)
predict_scores = base_model.predict(test["MovieID"])

In [7]:
metrics.get_ml_metrics(true_scores, predict_scores)

{'mae': 0.785, 'rmse': 0.985, 'precision': 0.805, 'recall': 0.361}